#  Segmenting and Clustering Neighborhoods in Toronto

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe 

To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
 * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
 * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.
 * These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
 * If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
 * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
 * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [2]:
# install libraries
!pip install beautifulsoup4
!pip install lxml

# import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [3]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

In [4]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)


# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        
df.head()     


,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:

#process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df=df[df['Borough']!='Not assigned']

# For neighborhoods sharing a postal code, combine them into one row with the neighborhoods separated with a comma .
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)

#If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
df.head(10)


,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
df_coord = pd.read_csv("Geospatial_Coordinates.csv") 
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_toronto= pd.merge(df, df_coord,how='left', left_on = 'Postalcode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
